In [15]:
import sys
sys.path.append("../../")
import os
os.environ["CUDA_VISIBLE_DEVICES"]="5"
import yaml
import numpy as np
from utils import *
from utils import save as usave
from torchvision import transforms
from trainer.dataset import FrameDataset
from trainer.models import ResModel
import pickle5 as pickle
import pdb
from torch.utils.data import DataLoader
import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

def load_config():
    with open("config_cnn.yaml", "r") as configfile:
        config_dict = yaml.load(configfile, Loader=yaml.FullLoader)
    # print(config_dict)
    return config_dict

In [7]:
config_dict = load_config()
    
if(config_dict['global']['enable_wandb']):
    import wandb
    wandb.init(name=config_dict['global']['iteration'], 
        project="AIGD", 
        notes=config_dict['global']['description']) 
    #  , config=config_dict)
else:
    wandb = None

print(config_dict)

cuda = torch.cuda.is_available()
print(cuda)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
epochs = config_dict['trainer']['epochs']

{'global': {'root_dir': '', 'iteration': 'library_cnn_resnet', 'description': 'indoor_baseline full run NO intent, no sampling', 'enable_preprocessing': False, 'seed': 8, 'enable_intent': False, 'enable_wandb': False}, 'transformer': {'fps': 2, 'path': '../../data/', 'data_save_file': '../../data/train', 'enable_benchmark_test': True, 'test_path': '../../data/', 'test_save_file': '../../data/test'}, 'data': {'HEIGHT': 64, 'WIDTH': 64, 'CHANNELS': 1, 'SEQUENCE_LENGTH': 10, 'BENCHMARK_TEST_INTENT': ''}, 'trainer': {'BATCH': 32, 'lr': 0.0001, 'epochs': 20, 'lambda': 0.001, 'num_classes': 3, 'model_save_path': '../../models/', 'model': {'name': 'resnet', 'fixconvs': True, 'pretrained_path': '', 'optimizer_path': ''}}}
True


In [8]:
df_videos = dict(np.load(config_dict['transformer']['data_save_file'] + '_video.npz', allow_pickle=True))
print("Videos in the training file: ", df_videos.keys())

# need video and sensor data separately
with open(config_dict['transformer']['data_save_file'] + '_sensor.pickle', 'rb') as handle:
    df_sensor = pickle.load(handle)

if (config_dict['transformer']['enable_benchmark_test'] == True):
    test_videos = dict(np.load(config_dict['transformer']['test_save_file'] + '_video.npz', allow_pickle=True))
    with open(config_dict['transformer']['test_save_file'] + '_sensor.pickle', 'rb') as handle:
        test_sensor = pickle.load(handle)
    print("Test Files: ", test_videos.keys())

else:
    test_videos = None
    test_sensor = None

train_transforms = transforms.Compose([transforms.ToTensor()])
val_transforms = transforms.Compose([transforms.ToTensor()])

train_files, val_files = make_tt_split(list(df_videos.keys()),config_dict['global']['seed'])


Videos in the training file:  dict_keys(['2022-09-25T15:52:44', '2022-09-25T16:16:15', '2022-09-24T15:59:46', '2022-07-12T17-46-01', '2022-09-24T16:29:46', '2022-09-24T16:13:32', '2022-07-12T17-54-18', '2022-07-12T17-25-48', '2022-09-24T16:04:23', '2022-09-24T15:37:31', '2022-09-24T15:18:08', '2022-09-25T16:25:49', '2022-09-24T16:20:46', '2022-09-24T15:41:55', '2022-09-25T15:30:45', '2022-09-24T15:32:17', '2022-09-24T15:51:04', '2022-07-12T17-52-00', '2022-09-25T15:57:42', '2022-09-24T15:27:58', '2022-09-25T15:43:59', '2022-07-12T17-15-22', '2022-09-24T15:23:37', '2022-09-24T16:08:54', '2022-09-24T16:34:16', '2022-09-24T15:12:51', '2022-07-12T16-52-14', '2022-09-25T15:48:21', '2022-07-12T16-44-21', '2022-09-25T16:21:25', '2022-09-24T16:25:22', '2022-07-12T17-08-12', '2022-07-12T17-02-16', '2022-07-12T17-12-05', '2022-07-12T17-38-36', '2022-07-12T17-32-15', '2022-09-24T15:46:25', '2022-09-25T15:39:29', '2022-09-24T16:42:30', '2022-09-25T15:35:09'])
Test Files:  dict_keys(['2022-07-12T17

In [9]:
train_dataset = FrameDataset(df_videos, df_sensor, sorted(train_files), transforms=train_transforms, config_dict=config_dict)
val_dataset = FrameDataset(df_videos, df_sensor, sorted(val_files), transforms=val_transforms, config_dict=config_dict)

train_args = dict(shuffle=True, batch_size=config_dict['trainer']['BATCH'], pin_memory=True, drop_last=False) if cuda else dict(batch_size=config_dict['trainer']['BATCH'], shuffle=True, drop_last=False)
train_loader = DataLoader(train_dataset, **train_args)

val_args = dict(shuffle=False, batch_size=config_dict['trainer']['BATCH'], num_workers=2, pin_memory=True, drop_last=False) if cuda else dict(shuffle=False, batch_size=config_dict['trainer']['BATCH'], drop_last=False)
val_loader = DataLoader(val_dataset, **val_args)

print("Length of val set: ", len(val_dataset))
print("Length of train set: ", len(train_dataset))

if config_dict['transformer']['enable_benchmark_test'] and test_videos is not None:
    test_dataset = FrameDataset(test_videos, test_sensor, sorted(list(test_videos.keys())), transforms=val_transforms, config_dict=config_dict)
    test_args = dict(shuffle=False, batch_size=config_dict['trainer']['BATCH'], num_workers=2, pin_memory=True, drop_last=False) if cuda else dict(shuffle=False, batch_size=config_dict['trainer']['BATCH'], drop_last=False)
    test_loader = DataLoader(test_dataset, **test_args)
    print("Length of test set: ", len(test_dataset))


Length of val set:  4523
Length of train set:  13981
Length of test set:  2308


In [11]:
model = ResModel(fixconvs = config_dict['trainer']['model']['fixconvs'])
model = model.to(device)
print(model)

weights = [2.0,2.0,1.0]
class_weights = torch.FloatTensor(weights).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=config_dict['trainer']['lr'], weight_decay=config_dict['trainer']['lambda'])
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=(len(train_loader) * epochs))


Local Resnet Lol
ResModel(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_run

In [16]:
from tqdm import tqdm
def train(epoch):
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
    
    num_correct = 0.0
    total_loss = 0.0
    y_cnt = 0.0
    actual = []
    predictions = []
    
    for _, (x, y) in enumerate(train_loader):
        model.train()
        optimizer.zero_grad()
        
        # print(x.shape)
        # print(y.shape)
        x = x.float().to(device)
        y = y.to(device)
        
        outputs = model(x)
        del x
        loss = criterion(outputs, y.long())

        pred_class = torch.argmax(outputs, axis=1)

        actual.extend(y.detach().cpu())
        predictions.extend(pred_class.detach().cpu())

        num_correct += int((pred_class == y).sum())
        del outputs
        total_loss += (float(loss)*len(y))
        y_cnt += len(y)

        batch_bar.set_postfix(
            acc="{:.04f}%".format(100 * float(num_correct) / y_cnt),
            loss="{:.04f}".format(float(total_loss) / y_cnt),
            num_correct=num_correct,
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
        
      
        loss.backward()
        optimizer.step()

        scheduler.step()
        batch_bar.update() # Update tqdm bar  
        

    batch_bar.close()
    total_loss = float(total_loss) / len(train_dataset)
    acc = 100 * float(num_correct) / (len(train_dataset))
    print("Epoch {}/{}: Train Acc {:.04f}%, Train Loss {:.04f}, Learning Rate {:.04f}".format(
        epoch + 1,
        epochs,
        acc,
        float(total_loss),
        float(optimizer.param_groups[0]['lr'])))

    if(wandb is not None):
        wandb.log({"Train Loss": total_loss, "Train Accuracy": acc, "Learning Rate": float(optimizer.param_groups[0]['lr'])})


    return actual, predictions


def validate():
    model.eval()
    val_num_correct = 0

    actual = []
    predictions = []

    
    for _, (vx, vy) in tqdm(enumerate(val_loader)):
    
        vx = vx.to(device)
        vy = vy.to(device)

        with torch.no_grad():
            outputs = model(vx)
            del vx

        pred_class = torch.argmax(outputs, axis=1)

        actual.extend(vy.detach().cpu())
        predictions.extend(pred_class.detach().cpu())

        val_num_correct += int((pred_class == vy).sum())
        
        del outputs
        

    acc = 100 * float(val_num_correct) / (len(val_dataset))
    print("Validation: {:.04f}%".format(acc))

    if(wandb is not None):
        wandb.log({"Validation Accuracy": acc})
    
    return acc, actual, predictions


# runs benchmark test at the end (after train and validation)
def test():
    model.eval()
    val_num_correct = 0

    actual = []
    predictions = []

    for i, (vx, vy) in tqdm(enumerate(test_loader)):
        vx = vx.to(device)
        vy = vy.to(device)

        with torch.no_grad():
            outputs = model(vx)
            del vx

        pred_class = torch.argmax(outputs, axis=1)

        actual.extend(vy.detach().cpu())
        predictions.extend(pred_class.detach().cpu())

        val_num_correct += int((pred_class == vy).sum())
        del outputs
        
    acc = 100 * float(val_num_correct) / (len(test_dataset))
    print("Benchmark test: {:.04f}%".format(acc))

    return acc, actual, predictions


def trainer_save(acc, epoch):
    usave(config_dict, model, epoch, acc, optim = False)
    usave(config_dict, optimizer, epoch, acc, optim = True)
    

In [17]:
trainer_save(0, -1)

In [18]:
for epoch in range(epochs):
    train_actual, train_predictions = train(epoch)
    acc, val_actual, val_predictions = validate()
    val_precision, val_recall, val_f1, _ = display_classification_report(train_actual, train_predictions, val_actual, val_predictions)
    
    if(config_dict['global']['enable_wandb']):
        wandb.log({"Val Precision 0": val_precision[0], "Val Precision 1": val_precision[1], "Val Precision 2": val_precision[2]})
        wandb.log({"Val Recall 0": val_recall[0], "Val Recall 1": val_recall[1], "Val Recall 2": val_recall[2]})
        wandb.log({"Val F1 0": val_f1[0], "Val F1 1": val_f1[1], "Val F1 2": val_f1[2]})

    trainer_save(acc, epoch)
    

print("Completed Training!!")
# performs final benchmarking after training
if (config_dict['transformer']['enable_benchmark_test'] == True):
    print("Starting benchmark testing!!")
    acc, test_actual, test_predictions = test()
    test_precision, test_recall, test_f1, _ = display_test_classification_report(test_actual, test_predictions)
    if(config_dict['global']['enable_wandb']):
        wandb.log({"Test Precision 0": test_precision[0], "Test Precision 1": test_precision[1], "Test Precision 2": test_precision[2]})
        wandb.log({"Test Recall 0": test_recall[0], "Test Recall 1": test_recall[1], "Test Recall 2": test_recall[2]})
        wandb.log({"Test F1 0": test_f1[0], "Test F1 1": test_f1[1], "Test F1 2": test_f1[2]})

    

Train:   0%|          | 0/437 [00:00<?, ?it/s]

RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[32, 1, 64, 64] to have 3 channels, but got 1 channels instead